<a href="https://colab.research.google.com/github/zahra-zarrabi/FaceRecognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

configs = {
              "learning_rate": 0.001,
              "epochs": 50,
              "batch_size": 32
           }

wandb.login()
run = wandb.init(project='face-recognition', config=configs)
config = wandb.config


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **👩‍🍳 Prepare Dataset**

In [ ]:
width=height=256
image_data_generator = ImageDataGenerator(rescale=1./255,
                                          horizontal_flip = True,
                                          validation_split= 0.2)

train_data = image_data_generator.flow_from_directory('/content/drive/MyDrive/7-7 dataset',
                                                      class_mode='categorical',
                                                      batch_size=config.batch_size,
                                                      shuffle=True,
                                                      target_size=(width,height),
                                                      subset='training')

val_data = image_data_generator.flow_from_directory('/content/drive/MyDrive/7-7 dataset',
                                                      class_mode='categorical',
                                                      batch_size=config.batch_size,
                                                      shuffle=False,
                                                      target_size=(width, height),
                                                      subset='validation')

Found 1091 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


In [ ]:
train_images = next(train_data)[0]
plt.figure(figsize = (8,8))
for i in range(16):
  plt.subplot(4,4,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  # plt.imshow(train_images[i])
  print(train_images.shape)

# 🧠 Define the Model and the Training Loop

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self,number_of_classes):
    super().__init__()
    self.conv1=Conv2D(32,(3,3),input_shape=(width,height,3),activation='relu')
    self.conv2=Conv2D(64,(3,3),activation='relu')
    self.conv3=Conv2D(128,(3,3),activation='relu')
    # self.conv4=Conv2D(128,(3,3),activation='relu') 
    self.maxpooling = MaxPooling2D()
    self.flatten=Flatten()
    self.dropout = Dropout(0.5)
    self.fc1=Dense(256,activation='relu')
    self.fc2=Dense(128,activation='relu')
    self.fc3=Dense(number_of_classes, activation='softmax')
    
  def call(self,x):    
    x = self.conv1(x)
    x = self.maxpooling(x)
    x = self.conv2(x)
    x = self.maxpooling(x)
    x = self.conv3(x)
    x = self.maxpooling(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = self.dropout(x)
    output = self.fc3(x)
    return output

In [ ]:
model.summary()

In [ ]:
model = MyModel(14)

In [ ]:
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name='train_acc')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_acc = tf.keras.metrics.CategoricalAccuracy(name='val_acc')


In [ ]:
def train_step(images,y):
  with tf.GradientTape() as tape:
    y_pred = model(images,training=True)    
    loss = loss_function(y,y_pred)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_acc(y,y_pred)
 

In [ ]:
def val_step(images,y):
  y_pred = model(images,training=False)
  loss = loss_function(y,y_pred)

  val_loss(loss)
  val_acc(y,y_pred)

In [ ]:
def train():
  epochs=config.epochs
  for epoch in range(epochs):

    train_loss.reset_states()
    train_acc.reset_states()
    val_loss.reset_states()
    val_acc.reset_states()

    cnt = 0
    for cnt in tqdm(range(train_data.__len__())):
      images,labels = next(train_data)
      train_step(images,labels)
    
    for cnt in tqdm(range(val_data.__len__())):
      images,labels = next(val_data)
      val_step(images,labels)

    print('epoch:',epoch+1)
    print('loss:',train_loss.result())
    print('accuracy:',train_acc.result())
    print('val loss:',val_loss.result())
    print('val accuracy:',val_acc.result())
    print('-------------------------------------------------------------------')


    #log metrics using wandb.log
    wandb.log({'epochs': epoch,
                'loss': train_loss.result(),
                'acc': train_acc.result(), 
                'val_loss': val_loss.result(),
                'val_acc':val_acc.result()})

In [ ]:
#fit
train()

100%|██████████| 9/9 [00:28<00:00,  3.21s/it]


epoch: 1
loss: tf.Tensor(2.9224, shape=(), dtype=float32)
accuracy: tf.Tensor(0.06874427, shape=(), dtype=float32)
val loss: tf.Tensor(2.6066816, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.15298508, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 9/9 [00:05<00:00,  1.79it/s]


epoch: 2
loss: tf.Tensor(2.5360038, shape=(), dtype=float32)
accuracy: tf.Tensor(0.14757104, shape=(), dtype=float32)
val loss: tf.Tensor(2.3808403, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.29850745, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 9/9 [00:04<00:00,  1.83it/s]


...

epoch: 48
loss: tf.Tensor(0.062783405, shape=(), dtype=float32)
accuracy: tf.Tensor(0.984418, shape=(), dtype=float32)
val loss: tf.Tensor(0.68363255, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.8768657, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 9/9 [00:05<00:00,  1.77it/s]


epoch: 49
loss: tf.Tensor(0.045497995, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9862512, shape=(), dtype=float32)
val loss: tf.Tensor(0.86530155, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.86567163, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 9/9 [00:05<00:00,  1.77it/s]

epoch: 50
loss: tf.Tensor(0.066091955, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9862512, shape=(), dtype=float32)
val loss: tf.Tensor(0.71518904, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.880597, shape=(), dtype=float32)
-------------------------------------------------------------------


In [ ]:
model.save_weights('/content/drive/MyDrive/model2/')